## 1. Load the Schema Names From the TableName text file.

In [3]:
with open('data/output_data/TableNames.txt', 'r') as textfile:
    schemas_to_convert = [line.split('.')[0] for line in textfile]

schemas_to_convert

['CUSTOMERS_SCHEMA',
 'RANDOM_SCHEMA1',
 'PRODUCT_REVIEWS_SCHEMA',
 'RANDOM_SCHEMA3',
 'RANDOM_SCHEMA4',
 'PRODUCTS_SCHEMA',
 'PURCHASES_SCHEMA',
 'CATEGORIES_SCHEMA',
 'RANDOM_SCHEMA2']

## 2. Read the SQL File Content

In [4]:
with open('data/SampleComplexQuery.sql', 'r') as sqlfile:
    sql_content = sqlfile.read()

## 3. Using `re.sub` Replace Schemas

In [6]:
import re

sample_sql = """SELECT * FROM CUSTOMERS_SCHEMA.customers WHERE customer_name LIKE '%Shannon%'; """
pattern = r"\b("+ "|".join(schemas_to_convert) + r")\."
new_sql = re.sub(pattern, "STAGING_SCHEMA.", sample_sql)
new_sql

"SELECT * FROM STAGING_SCHEMA.customers WHERE customer_name LIKE '%Shannon%'; "

**1. Regular Expression:**

* **`r"\b(" + "|".join(schemas_to_convert) + r")\."`:** This is the heart of the pattern matching and replacement.
    * `r":` defines a raw string, preventing escape characters from being interpreted literally.
    * `\b`: Matches word boundaries, preventing partial matches within words (e.g., "account" won't match "customer_account").
    * `(`: Starts a capturing group.
    * `|`: Pipe operator separates alternative patterns within the group.
    * `" + "|".join(schemas_to_convert) + "` dynamically inserts each schema name from the list, creating a pattern matching any of them.
    * `")\."`: Matches the captured schema name followed by a dot.

**2. Replacement String:**

* `"`: Starts and ends the replacement string.
* `"STAGING_SCHEMA."`: The replacement string containing the desired target schema.

**3. Flags:**

* `re.IGNORECASE`: Optional flag set based on `case_sensitive` argument. Makes the matching case-insensitive.
